In [1]:
import csv

# Path to your CSV file
csv_file_path = "Poem_Dataset.csv"

# Open and count rows
with open(csv_file_path, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    rows = list(reader)
    print(f"Total number of rows (including header): {len(rows)}")
    print(f"Total number of poems (excluding header): {len(rows) - 1}")


Total number of rows (including header): 2561
Total number of poems (excluding header): 2560


In [54]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException
import time

# Keywords to filter titles
keywords = ["xuân", "hạ", "thu", "đông", "núi", "biển","hoa", "lá"]
#["quê", "quê hương", "đất nước", "tổ quốc", "nước", "đất"]
#["mẹ", "cha", "chị", "vợ", "chồng"]
#["anh", "em", "tình yêu"]

# Configure Edge WebDriver
edge_options = Options()
# edge_options.add_argument("--headless")  # Uncomment for headless mode
edge_options.add_argument("--disable-gpu")
service = Service(executable_path="C:/Users/Hieu/Downloads/edgedriver_win64/msedgedriver.exe")

# Initialize WebDriver
driver = webdriver.Edge(service=service, options=edge_options)

# Open the main page
main_page_url = "http://vietnamthuquan.eu/Tho/?tranghientai=644"
driver.get(main_page_url)
time.sleep(3)  # Allow the main page to load

# CSV file setup
output_file = "poems_final.csv"

# Check if the file exists and set the writing mode
write_mode = "a"  # Append mode
file_header = False  # Flag to write the header only if the file is empty
try:
    with open(output_file, "r", encoding="utf-8") as f:
        if not f.read().strip():
            file_header = True
except FileNotFoundError:
    file_header = True

# Open the file for appending
with open(output_file, mode=write_mode, encoding="utf-8", newline="") as file:
    writer = csv.writer(file)

    # Write the header only if needed
    if file_header:
        writer.writerow(["title", "content"])

    try:
        current_page = 644  # Start 

        while True:
            print(f"Processing page {current_page}...")

            # Locate poem links on the current page
            try:
                poem_links = driver.find_elements(By.XPATH, "//*[@id='khungchinh']/div[5]/ul/div/li/a")
                print(f"Found {len(poem_links)} poem links on this page.")
            except Exception as e:
                print(f"Error locating poem links on page {current_page}: {e}")
                break

            # Process each poem
            for i in range(len(poem_links)):
                # Re-fetch links dynamically to avoid stale element reference
                poem_links = driver.find_elements(By.XPATH, "//*[@id='khungchinh']/div[5]/ul/div/li/a")
                link = poem_links[i]
                poem_title = link.text.strip()

                # Check if the title contains any of the keywords
                if any(keyword in poem_title.lower() for keyword in keywords):
                    print(f"Matching poem found: {poem_title}")

                    # Navigate to the poem page
                    poem_url = link.get_attribute("href")
                    driver.get(poem_url)
                    time.sleep(2)  # Allow the poem page to load

                    # Extract poem content
                    try:
                        # Locate all lines in the poem
                        poem_lines = driver.find_elements(By.XPATH,
                            "/html/body/div[2]/div[2]/div/div[1]/article/div/div/div/div[4]/div/div | "
                            "/html/body/div[2]/div[2]/div/div[1]/article/div/div/div/div[4]/div/p/span |"
                            "/html/body/div[2]/div[2]/div/div[1]/article/div/div/div/div[4]/div/p[1] |"
                            "/html/body/div[2]/div[2]/div/div[1]/article/div/div/div/div[4]/div |"
                            "/html/body/div[2]/div[2]/div/div[1]/article/div/div/div/div[4]/div/div/font/em |"
                            "/html/body/div[2]/div[2]/div/div[1]/article/div/div/div/div[4]/div/p/font"
                        )

                        # Filter out unwanted lines (e.g., author lines)
                        filtered_lines = [
                            line.text.strip()
                            for line in poem_lines
                            if line.text.strip()
                        ]
                        poem_content = "\n".join(filtered_lines)

                        if not poem_content:
                            raise ValueError("Poem content is empty.")

                        # Write to CSV
                        writer.writerow([poem_title, poem_content])
                        print(f"Extracted poem: {poem_title}")

                    except Exception as e:
                        print(f"Error extracting poem '{poem_title}': {e}")
                        writer.writerow([poem_title, "Error: Could not scrape content."])

                    # Return to the current page
                    driver.back()
                    time.sleep(3)  # Allow the page to reload

            # Click the "Next Page" button
            try:
                current_page += 1
                next_page_xpath = f"/html/body/div[1]/div[2]/div/div[1]/article/div/div/div/div[4]/h1/a[@href='?tranghientai={current_page}']"
                next_page_button = driver.find_element(By.XPATH, next_page_xpath)
                next_page_button.click()
                time.sleep(3)  # Allow the next page to load
            except NoSuchElementException:
                print(f"No more pages to navigate after page {current_page - 1}.")
                break

    except Exception as e:
        print(f"Error during execution: {e}")

    finally:
        driver.quit()
        print(f"Data saved to {output_file}")


Processing page 644...
Found 20 poem links on this page.
Matching poem found: BÀI HÁT VỀ NĂM CHIẾC LÁ
Extracted poem: BÀI HÁT VỀ NĂM CHIẾC LÁ
Matching poem found: MÙA TRĂNG HOÀNG HOA
Extracted poem: MÙA TRĂNG HOÀNG HOA
Matching poem found: MÙA THU LÁ RƠI
Extracted poem: MÙA THU LÁ RƠI
Processing page 645...
Found 20 poem links on this page.
Matching poem found: BIỂN RIÊNG EM
Extracted poem: BIỂN RIÊNG EM
Processing page 646...
Found 20 poem links on this page.
Matching poem found: KHÚC MÙA THU
Extracted poem: KHÚC MÙA THU
Processing page 647...
Found 20 poem links on this page.
Matching poem found: ĐÂU RỒI NHỮNG CÁNH HOA
Extracted poem: ĐÂU RỒI NHỮNG CÁNH HOA
Matching poem found: DÒNG SÔNG VÀ HOA LỘC VỪNG!
Extracted poem: DÒNG SÔNG VÀ HOA LỘC VỪNG!
Processing page 648...
Found 20 poem links on this page.
Matching poem found: BIỂN CẠN
Extracted poem: BIỂN CẠN
Matching poem found: BÔNG HOA NHỎ
Extracted poem: BÔNG HOA NHỎ
Processing page 649...
Found 20 poem links on this page.
Matching 

KeyboardInterrupt: 

In [58]:
import csv

# Input and output file paths
input_file = "poems_cleaned.csv"
output_file = "poems_thiên_nhiên.csv"

# Open the input file for reading and create a new output file for writing
with open(input_file, mode="r", encoding="utf-8") as infile, open(output_file, mode="w", encoding="utf-8", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # Read the header from the input file and add a new column "Category"
    header = next(reader)
    header.append("Category")
    writer.writerow(header)
    
    # Add "Thơ gia đình" as the category for all rows
    for row in reader:
        row.append("Thơ thiên nhiên")
        writer.writerow(row)

print(f"New file with category column saved to {output_file}")


New file with category column saved to poems_thiên_nhiên.csv


In [56]:
import csv

# Input and output file paths
input_file = "poems_final.csv"
output_file = "poems_cleaned.csv"

# Open the input file for reading and output file for writing
with open(input_file, mode="r", encoding="utf-8") as infile, open(output_file, mode="w", encoding="utf-8", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # Write the header row to the output file
    header = next(reader)
    writer.writerow(header)
    
    # Filter out rows where the content is "Error: Could not scrape content."
    for row in reader:
        # Skip empty rows or rows with fewer columns than expected
        if len(row) < 2:
            continue
        
        title, content = row
        if content.strip() != "Error: Could not scrape content.":
            writer.writerow(row)

print(f"Cleaned data saved to {output_file}")


Cleaned data saved to poems_cleaned.csv


In [45]:
import pandas as pd

file_paths = [
    "Dataset/Thơ_đất_nước.csv",
    "Dataset/Thơ_gia_đình.csv",
    "Dataset/Thơ_thiên_nhiên.csv",
    "Dataset/Thơ_tình.csv"
]

# Combine
dataframes = [pd.read_csv(file_path) for file_path in file_paths]
combined_dataset = pd.concat(dataframes, ignore_index=True)

# Save
output_file = "Poem_Dataset.csv"
combined_dataset.to_csv(output_file, index=False)

print(f"File tổng hợp đã được lưu tại: {output_file}")


File tổng hợp đã được lưu tại: Combined_Poem_Dataset.csv


In [2]:
import pandas as pd

combined_file_path = "Processed_Poem_Dataset.csv"

combined_dataset = pd.read_csv(combined_file_path)

# Count the number of poems of each category
class_counts = combined_dataset['Category'].value_counts()

# Number of poems in each category
print("Số lượng bài thơ của từng class:")
for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} bài thơ")


Số lượng bài thơ của từng class:
Class 'Thơ gia đình': 600 bài thơ
Class 'Thơ thiên nhiên': 600 bài thơ
Class 'Thơ tình': 600 bài thơ
Class 'Thơ đất nước': 600 bài thơ


In [ ]:
import pandas as pd

# Load the dataset from the provided file path
file_path = 'Processed_Poem_Dataset.csv'
dataset = pd.read_csv(file_path)

# Randomly sample 600 poems from each category
expanded_balanced_dataset = dataset.groupby('Category').apply(lambda x: x.sample(n=600, random_state=42)).reset_index(drop=True)

# Save the expanded balanced dataset to a new file
expanded_output_path = 'Processed_Poem_Dataset.csv'
expanded_balanced_dataset.to_csv(expanded_output_path, index=False)

# Output file path
print(expanded_output_path)
